In [4]:
# !pip install --upgrade metDataModel # https://github.com/shuzhao-li/metDataModel/ 
# !pip install -r ./requirements.txt --upgrade

In [5]:
import cobra # https://cobrapy.readthedocs.io/en/latest/io.html#SBML
from metDataModel.core import Compound, Reaction, Pathway, MetabolicModel
import sys
import re

sys.path.append("/Users/gongm/Documents/projects/mass2chem/")
sys.path.append("/Users/gongm/Documents/projects/JMS/JMS/JMS")
from mass2chem.formula import *
from jms.formula import *
from jms.utils.gems import *
from jms.utils.git_download import *
import pandas as pd
import pickle
import os

from datetime import datetime

# Functions

In [6]:
# Somehow the function was not corrected in jms.utils.gems
# I use this function cover the bugged one.
def neutral_formula2mass(neutral_formula):
    '''
    Convert neutral formula to mass but removing characters (e.g., X, R) typical in GEM but not ready for metabolomics application
    '''
    formula_dict = parse_chemformula_dict(neutral_formula)
    if ("R" not in formula_dict) & ("X" not in formula_dict) & ("Z" not in formula_dict) & (len(formula_dict) != 0) :
        mono_mass = calculate_mass(formula_dict,6)
    else:
        mono_mass = float(0)
    return(mono_mass)

#### Port metabolite

In [7]:
def port_metabolite(M, annotation_as_dict = True):
    '''
    convert cobra Metabolite to metDataModel Compound
    Annotation needs to be converted to dictionary structure rather than list of tuples for mummichog
    '''
    Cpd = Compound()
    Cpd.src_id = remove_compartment_by_split(M.id,'_')
    Cpd.id = remove_compartment_by_split(M.id,'_')              # temporarily the same with the source id
    Cpd.name = M.name.rsplit('-',1)[0]
    Cpd.charge = M.charge
    Cpd.neutral_formula = adjust_charge_in_formula(M.formula,M.charge)
    try:
        Cpd.neutral_mono_mass = neutral_formula2mass(Cpd.neutral_formula)
    except:
        Cpd.neutral_mono_mass = 0
    Cpd.charged_formula = M.formula
    
    if annotation_as_dict == True:
        Cpd.db_ids = {}
        Cpd.db_ids[strain_name] = Cpd.src_id
    else:
        Cpd.db_ids = [[strain_name,Cpd.src_id]] # using src_id to also reference Rat-GEM ID in db_ids field
    for k,v in M.annotation.items():
        if k not in ['sbo','hmdb','kegg.compound']:
            if annotation_as_dict == True:
                Cpd.db_ids[k] = v
            else:
                if isinstance(v,list):
                    Cpd.db_ids.append([[k,x] for x in v])
                else: 
                    if ":" in v:
                        Cpd.db_ids.append([k,v.split(":")[1]])
                    else:
                        Cpd.db_ids.append([k,v])
                
    return Cpd
    Cpd.db_ids = listOfTuple2dict(tuple([tuple(x) for x in Cpd.db_ids]))
    
    inchi_list = [x[1].split('=')[1] for x in Cpd.db_ids if x[0] == 'inchi']
    if len(inchi_list) ==1:
        Cpd.inchi = inchi_list[0]
    elif len(inchi_list) >1:
        Cpd.inchi = inchi_list
        
    return Cpd

#### Port reactions

In [8]:
# port reactions, to include genes and enzymes
def port_reaction(R):
    new = Reaction()
    new.id = remove_compartment_by_split(R.id,'_')
    new.reactants = [remove_compartment_by_split(x.id,'_') for x in R.reactants]
    new.products = [remove_compartment_by_split(x.id,'_') for x in R.products]
    new.genes = [g.id for g in R.genes]
    new.enzymes = R.annotation.get('ec-code', [])
    if 'EX_' not in new.id:
        return new

#### Port pathway

In [9]:
# pathways, using group as pathway. Other models may use subsystem etc.

def port_pathway(P):
    new = Pathway()
    new.id = P.id
    new.source = ['gapseq',]
    new.name = P.name
    new.list_of_reactions = [remove_compartment_by_split(x.id,'_') for x in P.members]
    return new


#### Wrapper to export metabolic_model

In [10]:
def export_metabolic_model(local_path, file_name, output_fdr, stain_name):
    
    today =  str(datetime.today()).split(" ")[0]
    print(today)
    # Read the model via cobra
    model = cobra.io.read_sbml_model(os.path.join(local_path,file_name))
    
    # Extract metabolites
    myCpds = []
    for i in range(len(model.metabolites)):
        myCpds.append(port_metabolite(model.metabolites[i]))
    print(f"{len(myCpds)} compounds in the model {stain_name}")
    myCpds = remove_duplicate_cpd(myCpds) # remove duplicated compounds
    print(f"{len(myCpds)} deduplicated compounds in the model {stain_name}")
    
    # Extract reactions
    myRxns = []
    for R in model.reactions:
        temp_rxn = port_reaction(R) 
        if temp_rxn:
            myRxns.append(temp_rxn)
    print(f"{len(myRxns)} reactions in the model {stain_name}")
    myRxns = remove_duplicate_rxn(myRxns)
    print(f"{len(myRxns)} deduplicated reactions in the model {stain_name}")

    # Extract pathways
    myPathways = []
    for P in model.groups:
        myPathways.append(port_pathway(P))
    print(f"{len(myPathways)} subsystems in the model {stain_name}")

    # Create MetabolicModel object
    MM = MetabolicModel()
    MM.id = f'az_gapseq_{stain_name}_{today}' #
    MM.meta_data = {
                'strains': strain_name.split('-')[0],
                'version': '',
                'sources': [f'gapseq, retrieved {today}'], #
                'status': '',
                'last_update': today,  #
                'note': "gapseq decompartmentalized, with genes and ECs.",
            }

    MM.list_of_reactions = [R.__dict__ for R in  myRxns]
    MM.list_of_compounds = [C.serialize() for C in myCpds]
    MM.list_of_pathways = [C.serialize() for C in myPathways]
    # print( MM.list_of_pathways)

    # Export files
    export_pickle(os.path.join(output_fdr,f'{MM.id}.pickle'), MM)
    export_json(os.path.join(output_fdr,f'{MM.id}.json'), MM)
    export_table(os.path.join(output_fdr,f'{MM.id}_list_of_compounds.csv'), MM, 'list_of_compounds')
    export_table(os.path.join(output_fdr,f'{MM.id}_list_of_reactions.csv'), MM, 'list_of_reactions')
    export_table(os.path.join(output_fdr,f'{MM.id}_list_of_pathways.csv'), MM, 'list_of_pathways')
    print("export metabolic model {stain_name} successfully")

--------

--------

--------

In [11]:
# Batch run
local_path = '../input/output_gapseq/faa_files_157_subdividing/gap_filled_xml_wt_draft/'
output_fdr = '../output/gapSeq_Jsons_faa_files_157/'
file_names = [x for x in os.listdir(local_path) if ".xml"in x]
strainIDs = [x.replace(".xml","") for x in file_names]

for file_name in file_names:
    strain_name = file_name.replace(".xml","")
    export_metabolic_model(local_path, file_name, output_fdr, strain_name)

'S_aureus-c8_0032-D06' is not a valid SBML 'SId'.


2023-04-05
1893 compounds in the model S_aureus-c8_0032-D06
1698 deduplicated compounds in the model S_aureus-c8_0032-D06
2124 reactions in the model S_aureus-c8_0032-D06
1981 deduplicated reactions in the model S_aureus-c8_0032-D06
1158 subsystems in the model S_aureus-c8_0032-D06
export metabolic model {stain_name} successfully
2023-04-05


'S_epidermidis-c4_0019-C07' is not a valid SBML 'SId'.


1825 compounds in the model S_epidermidis-c4_0019-C07
1635 deduplicated compounds in the model S_epidermidis-c4_0019-C07
2037 reactions in the model S_epidermidis-c4_0019-C07
1886 deduplicated reactions in the model S_epidermidis-c4_0019-C07
1134 subsystems in the model S_epidermidis-c4_0019-C07
export metabolic model {stain_name} successfully
2023-04-05


'S_hominis-c47_0028-H01' is not a valid SBML 'SId'.


...

1778 compounds in the model S_hominis-c16_016-C08
1586 deduplicated compounds in the model S_hominis-c16_016-C08
1980 reactions in the model S_hominis-c16_016-C08
1823 deduplicated reactions in the model S_hominis-c16_016-C08
1101 subsystems in the model S_hominis-c16_016-C08
export metabolic model {stain_name} successfully
2023-04-05


'S_capitis-c3_0009-D01' is not a valid SBML 'SId'.


1799 compounds in the model S_capitis-c3_0009-D01
1601 deduplicated compounds in the model S_capitis-c3_0009-D01
2017 reactions in the model S_capitis-c3_0009-D01
1854 deduplicated reactions in the model S_capitis-c3_0009-D01
1116 subsystems in the model S_capitis-c3_0009-D01
export metabolic model {stain_name} successfully
2023-04-05


'S_aureus-c5_0009-B05' is not a valid SBML 'SId'.


1863 compounds in the model S_aureus-c5_0009-B05
1668 deduplicated compounds in the model S_aureus-c5_0009-B05
2041 reactions in the model S_aureus-c5_0009-B05
1895 deduplicated reactions in the model S_aureus-c5_0009-B05
1137 subsystems in the model S_aureus-c5_0009-B05


## Summary

This ports reactions, pathways and compounds. Gene and enzyme information is now included. 

The exported pickle can be re-imported and uploaded to Database easily.

This notebook, the pickle file and the JSON file go to GitHub repo (https://github.com/shuzhao-li/Azimuth).